In [3]:
import gurobipy as gp
from gurobipy import GRB, LinExpr

model = gp.Model("ProductMix")

# Define the variables
products = ["JS-57", "LV-1231", "MRAB-012", "LR-MH", "FQ-0002", "NT-OC", "SW-S001", "SW-FL008", 
            "GQL-PO-S2", "LV-1031", "MRAB-112", "MRAB-22", "MRAB-117"]

# Create a dictionary of product variables
product_vars = {}
for product in products:
    product_vars[product] = model.addVar(vtype=GRB.INTEGER, name=product)

# Define the coefficients
machine_time_coeffs = [1, 1.5, 1, 1.5, 1.5, 2, 2.5, 3, 2, 2.5, 2, 1, 1.5]
raw_material_coeffs = [1.2, 0.504, 1.128, 1, 0.52, 1.265, 0.601, 1.069, 0.515, 0.49, 1.025, 0.51, 0.51]
labour_coeffs = [0.67] * 13
selling_prices = [2297, 1970, 2340, 2806, 1970, 2470, 2033, 2280, 1990, 1967, 2183, 2070, 2050]
production_costs = [2189, 1834, 2102, 1950, 1845, 2214, 1855, 2042, 1843, 1834, 1958, 1837, 1837]
lost_production_costs = [35024, 29344, 33632, 31200, 29520, 35424, 29680, 32672, 29488, 29344, 31328, 29392, 29392]
disposal_fee = 10000

# Initialize the objective function
obj = LinExpr()

 # Define the objective
for product in products:
        obj += (product_vars[product] - 16) * (selling_prices[products.index(product)] 
                - production_costs[products.index(product)]) - disposal_fee - lost_production_costs[products.index(product)]

# Set the objective to maximize profit
model.setObjective(obj, GRB.MAXIMIZE)

# Define the constraints
model.addConstr(sum(machine_time_coeffs[i] * product_vars[products[i]] for i in range(13)) <= 20000, "Machine_Time")
model.addConstr(sum(raw_material_coeffs[i] * product_vars[products[i]] for i in range(13)) <= 7000, "Raw_Material")
model.addConstr(sum(labour_coeffs[i] * product_vars[products[i]] for i in range(13)) <= 6000, "Labour")
model.addConstr(product_vars["JS-57"] >= 216, "Minimum_JS-57")
model.addConstr(4 * product_vars["LV-1231"] - 6 * product_vars["MRAB-012"] == 0, "Ratio_LV-1231_MRAB-012")
model.addConstr(9 * product_vars["LR-MH"] - 1 * product_vars["FQ-0002"] == 0, "Ratio_LR-MH_FQ-0002")
model.addConstr(product_vars["NT-OC"] <= 616, "NT-OC_Limit")

# Production vs. Demand constraints
demands = {"JS-57": 376, "LV-1231": 924, "MRAB-012": 617, "LR-MH": 129, "FQ-0002": 1041, "NT-OC": 659, "SW-S001": 1031,
           "SW-FL008": 1661, "GQL-PO-S2": 817, "LV-1031": 1538,
           "MRAB-112": 893, "MRAB-22": 265, "MRAB-117": 315}

for product in products:
    model.addConstr(product_vars[product] <= demands[product], f"Production_vs_Demand_{product}")

# Optimize the model
model.optimize()

# Display the results
for product in products:
    if product_vars[product].X > 1e-6:
        print(f"{product}: {product_vars[product].X}")

print("Total Profit:", model.objVal)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 20 rows, 13 columns and 58 nonzeros
Model fingerprint: 0x375016e2
Variable types: 0 continuous, 13 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 9e+00]
  Objective range  [1e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+04]
Found heuristic solution: objective 846742.00000
Presolve removed 18 rows and 2 columns
Presolve time: 0.00s
Presolved: 2 rows, 11 columns, 22 nonzeros
Found heuristic solution: objective 1079416.0000
Variable types: 0 continuous, 11 integer (0 binary)
Found heuristic solution: objective 1080366.0000

Root relaxation: objective 1.169060e+06, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O